In [2]:
import distanceCorrelation as dc
import numpy as np
import math
import pandas as pd
import random
import sklearn.gaussian_process as gp
import heapq
import copy
import scipy.stats as sp
import sys
import subprocess
import inspect
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
import datetime
import networkx as nx


#https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

In [3]:
df = pd.read_excel("input.xlsx")
df.drop(df[df.Group!="Ctrl"].index, inplace=True)
df.drop("Group", axis=1, inplace=True)
print(df)

    Glycerol  2-Hydroxybutyrate  2-Hydroxyisovalerate  3-Hydroxybutyrate  \
0    15669.4              205.2                  56.5              361.5   
1    10084.3              220.9                  92.6              438.0   
2    12836.6              394.7                  93.9              861.2   
3    10520.2              293.0                 200.1             1309.9   
4    15809.9              573.9                 131.3              580.0   
5    12581.0              182.0                  56.8              214.0   
6    12276.4             1103.3                 323.0              422.2   
7    13939.5              488.6                 156.6              493.3   
8    11628.6              567.3                  85.5              377.5   
9    17668.2             1043.2                 178.5              594.8   
10   10293.1              230.4                  52.6             1039.2   
11    6676.8              190.9                  62.1              224.7   
12    8192.4

In [19]:
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

kernel1 = gp.kernels.DotProduct()
kernel2 = gp.kernels.RBF() + gp.kernels.WhiteKernel()
kernel= kernel1+kernel2

#models=[]
loss=[] #to be 2D array for all loss values
count=0.0

for molecule1 in df:
    count+=1
    print(str(round(count*100/len(df.columns),2)) + "% complete")
    #molecule1models=[]
    molecule1Loss=[] #1D array, loss values for single molecule
    for molecule2 in df:
        if molecule1==molecule2: #ensure we don't compare a molecule to itself
            molecule1Loss+=[0.0]
            #molecule1models+=NULL

        else:
            #Splitting into train and validation sets
            x = df[molecule1].values
            x, cvx = np.split(x, [int(.8*len(x))])

            x = x.reshape(-1,1) #turn to 2D array
            cvx = cvx.reshape(-1,1)

            y = df[molecule2]
            y, cvy = np.split(y, [int(.8*len(y))])

            model = gp.GaussianProcessRegressor(kernel=kernel).fit(x,y)
            molecule1Loss+=[model.score(cvx,cvy)]

    loss+=[molecule1Loss]

0.46% complete
0.92% complete
1.38% complete
1.83% complete
2.29% complete
2.75% complete
3.21% complete
3.67% complete
4.13% complete
4.59% complete
5.05% complete
5.5% complete
5.96% complete
6.42% complete
6.88% complete
7.34% complete
7.8% complete
8.26% complete
8.72% complete
9.17% complete
9.63% complete
10.09% complete
10.55% complete
11.01% complete
11.47% complete
11.93% complete
12.39% complete
12.84% complete
13.3% complete
13.76% complete
14.22% complete
14.68% complete
15.14% complete
15.6% complete
16.06% complete
16.51% complete
16.97% complete
17.43% complete
17.89% complete
18.35% complete
18.81% complete
19.27% complete
19.72% complete
20.18% complete
20.64% complete
21.1% complete
21.56% complete
22.02% complete
22.48% complete
22.94% complete
23.39% complete
23.85% complete
24.31% complete
24.77% complete
25.23% complete
25.69% complete
26.15% complete
26.61% complete
27.06% complete
27.52% complete
27.98% complete
28.44% complete
28.9% complete
29.36% complete
29.

In [20]:
#output all correlations above threshold to console

threshold = 0

count=0

for i in range(len(loss)):
    for j in range(len(loss[i])):
        if loss[i][j]>threshold:
            print(df.columns[i]+" -> "+df.columns[j]+ " ("+str(loss[i][j])+")")
            count+=1


print(str(count) + " molecules have a score above " + str(threshold))

Glycerol -> 2-Hydroxybutyrate (0.25119285482233533)
Glycerol -> 2-Hydroxyisovalerate (0.365512666742137)
Glycerol -> 4-Aminobutyrate (0.5599321338927777)
Glycerol -> Acetate (0.6977872328030384)
Glycerol -> Acetone (0.2864428618224868)
Glycerol -> ADP (0.061035904895304305)
Glycerol -> Anserine (0.6838088947625328)
Glycerol -> Choline (0.5580972277661737)
Glycerol -> Creatine (0.12917876419704521)
Glycerol -> Dimethylamine (0.0044588715604869655)
Glycerol -> Ethanolamine (0.6105066456307998)
Glycerol -> Fumarate (0.2165120064394982)
Glycerol -> Hypoxanthine (0.6121026591254737)
Glycerol -> Isobutyrate (0.5956858464841301)
Glycerol -> Lactate (0.29458554143134663)
Glycerol -> Malonate (0.11643620222447648)
Glycerol -> Methanol (0.29853915733982017)
Glycerol -> myo-Inositol (0.19461465206434558)
Glycerol -> N-Acetylaspartate (0.32750536755475324)
Glycerol -> Niacinamide (0.12281071925425024)
Glycerol -> O-Acetylcarnitine (0.40210715032068944)
Glycerol -> Pantothenate (0.14989567098225043

In [21]:
#output all correlations above threshold to file

threshold = 0
count=0

filename = ("output/"+kernel1.__class__.__name__+"*"+kernel2.__class__.__name__+"_"+str(datetime.datetime.now())+".csv")
f=open(filename,"w")

for i in range(len(loss)):
    for j in range(len(loss[i])):
        if loss[i][j]>threshold:
            #molecule1, molecule2, correlation score
            #where molecule1 can be used to predict molecule2
            f.write(df.columns[i]+", "+df.columns[j]+ ", "+str(loss[i][j])+"\n")
            count+=1

f.close()

In [33]:

import networkx as nx
df = pd.read_csv("output/float*RBF_2023-03-10 13:41:09.517129.csv")
G = nx.from_pandas_edgelist(df, source="source", target="target")

nx.draw(G)

KeyError: 'target'